In [15]:
from datasets import load_dataset,concatenate_datasets
from tqdm import tqdm
import random
from datetime import datetime
import json
import os

os.system("mkdir -p out_data")
current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")
out_path = f"out_data/model_{current_time_no_symbols}.jsonl"


from vllm import LLM
import string
from vllm import SamplingParams


In [ ]:
model_name="microsoft/Phi-3-medium-128k-instruct"
#model_name="OrionStarAI/Orion-14B-Chat"
llm = LLM(model=model_name,trust_remote_code=True,
          max_model_len=20000
          )

In [18]:

#ds=load_dataset("kanhatakeyama/ChatbotArenaJaMixtral8x22b", split="train")
#ds=load_dataset("wikipedia", "20220301.en",streaming=True,split="train")
#ds=load_dataset("hpprc/jawiki-wiktionary", split="train")

streaming=True
ds_list=[
load_dataset("HuggingFaceTB/cosmopedia","auto_math_text",streaming=streaming,split="train"),
load_dataset("HuggingFaceTB/cosmopedia","khanacademy",streaming=streaming,split="train"),
load_dataset("HuggingFaceTB/cosmopedia","openstax",streaming=streaming,split="train"),
load_dataset("HuggingFaceTB/cosmopedia","stanford",streaming=streaming,split="train"),
load_dataset("HuggingFaceTB/cosmopedia","wikihow",streaming=streaming,split="train"),
]
ds=concatenate_datasets(ds_list)

In [16]:
combined=concatenate_datasets([ds,ds])

In [17]:
next(iter(combined))

{'text_token_length': 931,
 'prompt': 'Write a long and very detailed tutorial on "Compost", in the style of WikiHow. Include in depth explanations for each step and how it helps achieve the desired outcome, inluding key tips and guidelines. Ensure clarity and practicality, allowing readers to easily follow and apply the instructions. Do not use images.',
 'text': " Title: How to Create and Maintain a Compost Pile\n\nIntroduction:\nComposting is an easy and environmentally friendly way to recycle organic materials and create nutrient-rich soil for your garden or plants. By following these steps, you can learn how to build and maintain a successful compost pile that will help reduce waste and improve the health of your plants.\n\n**Step 1: Choose a Location **\nSelect a well-draining spot in your backyard, away from your house or other structures, as compost piles can produce odors. Ideally, locate the pile in partial shade or a location with morning sun only. This allows the pile to re

In [12]:
try:
    ds.shuffle()
except:
    pass

In [13]:
import random

inst_dict={

"textbook":"""次のデータをもとに､論理的かつ教科書調の丁寧な日本語の文章を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
""",

"conversation":"""次のデータをもとに､論理的な日本語の会話文を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
""",
"logical":"""次のデータをもとに､論理的な文章を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
"""
}
def extract_random_part(text):
    text_length = len(text)
    extract_length = min(text_length, random.randint(400, 2000))
    start_index = random.randint(0, text_length - extract_length)
    return text[start_index:start_index + extract_length]

mode_list=list(inst_dict.keys())
n_records=5


In [14]:
cnt=0
loader=ds
def prepare_records(loader,n_records=10):

    records=[]
    cnt=0
    for record in loader:
        mode=random.choice(mode_list)
        inst=inst_dict[mode]
        text=record["text"]
        text=extract_random_part(text)
        text=f"""<|user|>
    {inst}{text}<|end|>
    <|assistant|>"""
        records.append(
            {"original_text":text,
            "mode":mode,
            "url":record["url"]
            }
                    )
        cnt+=1
        if cnt>n_records:
            break

    return records

In [15]:
while True:
    records=prepare_records(loader,n_records)
    prompts=[record["original_text"] for record in records]
    outputs = llm.generate(
            prompts,
            sampling_params = SamplingParams(
                temperature=0.1,
                max_tokens=1024,
                repetition_penalty=1.2,
            )
        )

    for record,output in zip(records,outputs):
        record["output_text"]=(output.outputs[0].text).strip()
        record.pop("original_text")
        with open(out_path, "a") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")












Processed prompts: 100%|██████████| 12/12 [00:23<00:00,  1.98s/it]







Processed prompts: 100%|██████████| 6/6 [00:21<00:00,  3.59s/it]







Processed prompts: 100%|██████████| 6/6 [00:18<00:00,  3.06s/it]







Processed prompts: 100%|██████████| 6/6 [00:22<00:00,  3.83s/it]







Processed prompts: 100%|██████████| 6/6 [00:17<00:00,  2.84s/it]







Processed prompts: 100%|██████████| 6/6 [00:11<00:00,  1.94s/it]








KeyboardInterrupt: 